In [ ]:
question = ""

### **TF-IDF**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('item_pool.csv')

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the vectorizer
X = vectorizer.fit_transform(df['question'])
Y = vectorizer.transform([question])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(Y, X)
cosine_sim = cosine_sim.flatten()

# 코사인 유사도를 데이터프레임에 추가
df['cosine_sim'] = cosine_sim

# 성능 평가
tfidf_df = df.sort_values(by='cosine_sim', ascending=False).head(5)
tfidf = len(tfidf_df[tfidf_df['chapter']==chapter])
tfidf_score = tfidf/5
tfidf_score

### **Word2Vec** ##

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

sentences = [question.split() for question in questions]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_sentence_vector(x, model))

# input question을 벡터로 변환
question_vector = get_sentence_vector(question, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
word2vec_df = df.sort_values(by='cosine_sim', ascending=False)

word2vec_df.drop(columns=['vector'], inplace=True)

word2vec_df = word2vec_df.head(5)
word2vec = len(word2vec_df[word2vec_df['chapter']==chapter])
word2vec_score = word2vec/5
word2vec_score

### **Doc2Vec**

In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

# TaggedDocument로 변환
tagged_data = [TaggedDocument(words=question.split(), tags=[i]) for i, question in enumerate(questions)]

# Doc2Vec 모델 훈련
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4, epochs=40)

def get_doc2vec_vector(sentence, model):
    words = sentence.split()
    return model.infer_vector(words)

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_doc2vec_vector(x, model))

# input question을 벡터로 변환
question_vector = get_doc2vec_vector(question, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
doc2vec_df = df.sort_values(by='cosine_sim', ascending=False)

doc2vec_df.drop(columns=['vector'], inplace=True)

doc2vec_df = doc2vec_df.head(5)
doc2vec = len(doc2vec_df[doc2vec_df['chapter']==chapter])
doc2vec_score = doc2vec/5
doc2vec_score

### **KoBERT** ##

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

# KoBERT 로드
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertModel.from_pretrained('kykim/bert-kor-base')

def get_bert_vector(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_bert_vector(x, tokenizer, model))

# input question을 벡터로 변환
question_vector = get_bert_vector(question, tokenizer, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
bert_df = df.sort_values(by='cosine_sim', ascending=False)

bert_df.drop(columns=['vector'], inplace=True)

bert_df = bert_df.head(5)
bert_df
bert = len(bert_df[bert_df['chapter']==chapter])
bert_score = bert/5

In [44]:
score.append(tfidf_score)
score.append(word2vec_score)
score.append(doc2vec_score)
score.append(bert_score)

score

[0.4,
 0.8,
 0.2,
 0.4,
 0.8,
 0.4,
 0.2,
 0.8,
 0.6,
 0.8,
 0.2,
 0.6,
 0.8,
 0.6,
 0.0,
 0.4,
 0.0,
 0.2,
 0.0,
 0.0]

In [47]:
score_df = pd.DataFrame([score[4*i:4*(i+1)] for i in range(5)],
                  columns=['TF-IDF', 'Word2Vec', 'Doc2Vec', 'BERT'],
                  index=['Try1', 'Try2', 'Try3', 'Try4', 'Try5'])

column_means = score_df.mean()

score_df.loc['Average'] = column_means

score_df

,TF-IDF,Word2Vec,Doc2Vec,BERT
Try1,0.40,0.80,0.20,0.40
Try2,0.80,0.40,0.20,0.80
Try3,0.60,0.80,0.20,0.60
Try4,0.80,0.60,0.00,0.40
Try5,0.00,0.20,0.00,0.00
Average,0.52,0.56,0.12,0.44
